In [8]:
import json
import torch
from torch import nn
from copy import deepcopy
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [10]:
checkpoint = "models/checkpoint-71-1.44896/"
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint)
model = GPT2LMHeadModel.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token

In [11]:
valid = []
with open("data/process.valid.json") as fin:
    for line in fin.readlines():
        valid.append(json.loads(line))

In [12]:
def tokenize(data):
    res = tokenizer(data, max_length=256, truncation=True, padding='max_length', return_tensors='pt')
    res['labels'] = deepcopy(res['input_ids'])
    return res

In [13]:
data = [tokenize(x['text']) for x in valid]
valid_dataloader = DataLoader(data, batch_size=1)

In [14]:
model.eval();

In [15]:
for step, batch in enumerate(valid_dataloader):
    with torch.no_grad():
        outputs = model(**batch)
    loss = outputs.loss
    print(step, loss.item())

0 1.466373085975647
1 1.3611961603164673
2 1.5006167888641357
3 1.3286750316619873
4 1.2899988889694214
5 1.3993217945098877
6 1.34325110912323
7 1.6091742515563965
8 1.5511444807052612
9 1.4701207876205444
10 1.4888849258422852
11 1.5127161741256714
12 1.4671717882156372
13 1.5454691648483276
14 1.3994916677474976
15 1.5268012285232544
16 1.1971253156661987
17 1.2060883045196533
18 1.4378563165664673
19 1.2301539182662964
20 1.4487124681472778
21 1.3899378776550293
22 1.510952115058899
23 1.3130519390106201
24 1.7263386249542236
25 1.5806022882461548
26 1.3740794658660889
27 1.4311962127685547
28 1.7050901651382446
29 1.5969243049621582
30 1.2378023862838745
31 1.7410755157470703
32 1.3752601146697998
33 1.5047982931137085
34 1.5293471813201904
35 1.5695821046829224
36 1.6975607872009277
37 1.360790729522705
38 1.4324647188186646
39 1.5470514297485352
40 1.3905867338180542
41 1.2842612266540527


In [24]:
valid[16]

{'id': '13000',
 'text': '<sos_u> boa noite, pode me dizer quanto de saldo eu tenho? <eos_u><sos_b> [consulta_saldo][req_saldo][cumprimento] <eos_b><sos_a> [req_cpf] <eos_a><sos_r> Sim, Claro. Qual é o seu cpf? <eos_r><sos_u> 978403690-03 <eos_u><sos_b> [consulta_saldo][info_cpf] cpf 978403690-03 <eos_b><sos_a> [req_placa] <eos_a><sos_r> Me informe sua placa, por favor? <eos_r><sos_u> nxi 4451 <eos_u><sos_b> [consulta_saldo][info_placa] cpf 978403690-03 placa nxi 4451 <eos_b><sos_a> [info_valor][req_mais] <eos_a><sos_r> o saldo é de [valor], mais alguma coisa? <eos_r><sos_u> por enquanto nao. obrigado pelas confirmações <eos_u><sos_b> [negacao] cpf 978403690-03 placa nxi 4451 <eos_b><sos_a> [despedida] <eos_a><sos_r> até mais! <eos_r>'}

In [25]:
string = "<sos_u><sos_u> boa noite, pode me dizer quanto de saldo eu tenho? <eos_u><sos_b> [consulta_saldo][req_saldo][cumprimento] <eos_b><sos_a> [req_cpf] <eos_a><sos_r> Sim, Claro. Qual é o seu cpf? <eos_r><sos_u> 978403690-03 <eos_u><sos_b> [consulta_saldo][info_cpf] cpf 978403690-03 <eos_b><sos_a> [req_placa] <eos_a><sos_r> Me informe sua placa, por favor? <eos_r><sos_u> nxi 4451 <eos_u><sos_b> [consulta_saldo][info_placa] cpf 978403690-03 placa nxi 4451 <eos_b><sos_a> [info_valor][req_mais] <eos_a><sos_r> o saldo é de [valor], mais alguma coisa? <eos_r><sos_u> e para a placa jgm1234<eos_u><sos_b>"
input_ids = tokenizer.encode(string)
eos_token = tokenizer.encode(['<eos_r>'])[0]
out = model.generate(input_ids=torch.tensor(input_ids).reshape(1,-1),
                                 pad_token_id=tokenizer.eos_token_id,
                                 max_length=len(input_ids)+60,
                                 eos_token_id=eos_token)

In [26]:
tokenizer.decode(out[0])

'<sos_u> <sos_u> boa noite, pode me dizer quanto de saldo eu tenho? <eos_u> <sos_b> [consulta_saldo] [req_saldo] [cumprimento] <eos_b> <sos_a> [req_cpf] <eos_a> <sos_r> Sim, Claro. Qual é o seu cpf? <eos_r> <sos_u> 978403690-03 <eos_u> <sos_b> [consulta_saldo] [info_cpf] cpf 978403690-03 <eos_b> <sos_a> [req_placa] <eos_a> <sos_r> Me informe sua placa, por favor? <eos_r> <sos_u> nxi 4451 <eos_u> <sos_b> [consulta_saldo] [info_placa] cpf 978403690-03 placa nxi 4451 <eos_b> <sos_a> [info_valor] [req_mais] <eos_a> <sos_r> o saldo é de [valor], mais alguma coisa? <eos_r> <sos_u> e para a placa jgm1234 <eos_u> <sos_b> [negacao] cpf 978403690-03 placa jgm1234 <eos_b> <sos_a> [despedida] <eos_a> <sos_r> Obrigado pelo contato! Desejo uma boa tarde para você! Tchau! <eos_r>'